In [1]:
import numpy as np
import pandas as pd
import numpy_financial as npf

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.express as px

fv(rate, nper, pmt, pv, when) - Compute the future value.

pv(rate, nper, pmt, fv, when) - Compute the present value.

npv(rate, values) - Returns the NPV (Net Present Value) of a cash flow series.

pmt(rate, nper, pv, fv, when) - Compute the payment against loan principal plus interest.

ppmt(rate, per, nper, pv, fv, when) - Compute the payment against loan principal.

ipmt(rate, per, nper, pv, fv, when) - Compute the interest portion of a payment.

irr(values) - Return the Internal Rate of Return (IRR).

mirr(values, finance_rate, reinvest_rate) - Modified internal rate of return.

nper(rate, pmt, pv, fv, when) - Compute the number of periodic payments.

rate(nper, pmt, pv, fv, when, guess, tol, …) - Compute the rate of interest per period.

https://numpy.org/doc/1.19/reference/routines.financial.html

In [2]:
-npf.pmt(0.0365,27,0, 150000,0)

3353.715879660265

In [3]:
-npf.fv(0.0365,10,0,100000,0)

143117.60644172

In [4]:
-npf.pv(0.0365,27,0,150000,0)

56979.67731728589

## Loan

In [12]:
def big_loan(years,debt,r,
        years2,debt2,r2,
        years3,debt3,r3):

    Loan = debt
    Loan2 = debt2
    Loan3 = debt3
    
    debts = debt+debt2+debt3
    
    df = pd.DataFrame(columns=['year','pmt_m','pmt', 'keren','rate', 'debt'])

    df.loc[len(df.index)] = [0,None,None, 0,0,debts]
    
    tot_years = max(years,years2,years3)
    
    colors = ['gold', 'darkorange']

    for n in range(1,tot_years+1):
        
#         if n <=3:
#             inf = 0.03
#         elif n <= 5:
#             inf = 0.0
#         else:
#             inf = 0.0
        inf=0.02
        
        if n > years:
            pmt_m1=0
            pmt1=0
            keren1=0
            rate1=0
        else:
            pmt_m1 = -npf.pmt(r/12, years*12, Loan,0,0)
            pmt1 = -npf.pmt(r, years, Loan,0,0)
            keren1 = -npf.ppmt(r, n,years, Loan,0,0)
            rate1 = -npf.ipmt(r, n,years, Loan,0,0).item()
        if n > years2:
            pmt_m2=0
            pmt2=0
            keren2=0
            rate2=0
        else:
            pmt_m2 = -npf.pmt(r2/12, years2*12, Loan2,0,0)
            pmt2 = -npf.pmt(r2, years2, Loan2,0,0)
            keren2 = -npf.ppmt(r2, n,years2, Loan2,0,0)
            rate2 = -npf.ipmt(r2, n,years2, Loan2,0,0).item()
        if n > years3:
            pmt_m3 = 0
            pmt3 = 0
            keren3 = 0
            rate3 = 0
        else:
            pmt_m3 = -npf.pmt((r3+inf)/12, years3*12, Loan3,0,0)
            pmt3 = -npf.pmt((r3+inf), years3, Loan3,0,0) 
            keren3 = -npf.ppmt((r3+inf), n,years3, Loan3,0,0)
            rate3 = -npf.ipmt((r3+inf), n,years3, Loan3,0,0).item()
            
        pmt_m = round(pmt_m1+pmt_m2+pmt_m3,2)       
        pmt = round(pmt1+pmt2+pmt3,2)      
        keren = round(keren1+keren2+keren3,2)      
        rate = round(rate1+rate2+rate3,2)         
        
        debts = round(debts - keren,2)

        df.loc[len(df.index)] = [n,pmt_m,pmt,keren,rate,debts]
        
#Table Chart  
    table = ff.create_table(df, height_constant=60)
    table.update_layout(autosize=False,width=500,height=350)
    table.show()

# Pie chart
    pie = go.Figure(data=[go.Pie(labels=['Keren','Rate'],
                             values=[sum(df['keren']),sum(df['rate'])])])
    pie.update_layout(autosize=False,width=600,height=450,)
    pie.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=16,
                      marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    pie.show()
    
#Line Chart  
    subfig = make_subplots(specs=[[{"secondary_y": True}]])
    #Debt
    fig = px.line(df, x="year", y="debt",text="debt", markers=True)
    fig.update_layout(autosize=False,width=600,height=450)
    fig.update_traces(textposition="top right")

    #PMT
    fig2 = px.line(df, x="year", y="pmt_m", markers=True)
    fig2.update_traces(yaxis="y2")

    subfig.add_traces(fig.data + fig2.data)
    subfig.layout.title="Mortgage Progress" 
    subfig.layout.xaxis.title="Years"
    subfig.layout.yaxis.title="Total Debt"
    subfig.layout.yaxis2.title="Monthly PMT"

    # recoloring is necessary otherwise lines from fig und fig2 would share each color
    subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
    subfig.update_yaxes(range=[0,max(df['pmt_m'])*1.2], secondary_y=True)
    subfig.show()
    
# Stacked Bar Chart
    #prepare data for stacked bar chart
    keren_df = df[['year','keren']]
    keren_df['type'] = 'keren'
    keren_df = keren_df.rename(columns={"keren": "amount"})

    rate_df = df[['year','rate']]
    rate_df['type'] = 'rate'
    rate_df = rate_df.rename(columns={"rate": "amount"})

    mergio = pd.concat([keren_df,rate_df])
    mergio

    bar = px.bar(mergio, x="year", y="amount",
                 color='type',
                 barmode='group', #relative group
                 height=400)
    bar.update_layout(barmode='stack')
    bar.show()

In [13]:
# years,debt,r,

Kalaz =0.4
Praim = 0.35
Zmuda = 0.25

#D2021
# big_loan( 5,166000,0.017,
#          15,166000,0.01,
#          10,100000,0.0111)

# D2023
# big_loan( 4,120400,0.017,
#          4,42000,0.0515,
#          7,91800,0.0111)

# D2024
big_loan( 3,90300,0.017,
         0,0,0.0515,
         6,84900,0.0111)

#F
# big_loan(13,429030,0.0238,
#          27,270000,0.0116,
#          30,200000,0.0313)

#El
# big_loan(19,352917 ,0.0372,
#          19,166045 ,0.0085,
#          19, 59591 ,0.0362)

#Ed
# big_loan(30,523600 ,0.0365,
#          30,523600 ,0.0515,
#          30,261800 ,0.021)

#W
# big_loan(25,1000000*Kalaz,0.035,
#          25,1000000*Praim,0.0275,
#          25,1000000*Zmuda,0.03)

# QA

In [321]:
# Zamud
years = 10
debt = 100000
r = 0.0111
Loan = debt

df = pd.DataFrame(columns=['year','pmt_m','pmt', 'keren','rate', 'debt'])

df.loc[len(df.index)] = [0,0, 0, 0,0,debt]

for n in range(1,years+1):
    pmt_m = -round(npf.pmt(r/12, years*12, Loan,0,0),2)
    pmt = -round(npf.pmt(r, years, Loan,0,0),2)
    keren = -round(npf.ppmt(r, n,years, Loan+(debt*0.03),0,0),2)
    rate = -round(npf.ipmt(r, n,years, Loan+(debt*0.03),0,0).item(),2)
    debt = round(debt - keren,2)*1.03


    df.loc[len(df.index)] = [n,pmt_m,pmt,keren,rate,debt]
df

,year,pmt_m,pmt,keren,rate,debt
0,0,0,0,0,0,100000
1,1.0,880.82,10620.61,9795.92,1143.3,92910.2024
2,2.0,880.82,10620.61,9884.21,1032.43,85516.7697
3,3.0,880.82,10620.61,9972.36,920.72,77810.7423
4,4.0,880.82,10620.61,10060.32,808.2,69782.9326
5,5.0,880.82,10620.61,10148.05,694.9,61423.9264
6,6.0,880.82,10620.61,10235.49,580.82,52724.0932
7,7.0,880.82,10620.61,10322.58,466.01,43673.5553
8,8.0,880.82,10620.61,10409.27,350.49,34262.2187
9,9.0,880.82,10620.61,10495.48,234.29,24479.7422


## Maslul 1

In [202]:
def loan(years,debt,r):
    
    Loan = debt
    
    df = pd.DataFrame(columns=['year','pmt_m','pmt', 'keren','rate', 'debt'])

    df.loc[len(df.index)] = [0,0, 0, 0,0,debt]

    for n in range(1,years+1):
        pmt_m = -round(npf.pmt(r/12, years*12, Loan,0,0),2)
        pmt = -round(npf.pmt(r, years, Loan,0,0),2)
        keren = -round(npf.ppmt(r, n,years, Loan,0,0),2)
        rate = -round(npf.ipmt(r, n,years, Loan,0,0).item(),2)
        debt = round(debt - keren,2)


        df.loc[len(df.index)] = [n,pmt_m,pmt,keren,rate,debt]

#Table Chart
    table = ff.create_table(df, height_constant=60)
    table.update_layout(autosize=False,width=500,height=350)
    table.show()

# Pie chart
    pie = go.Figure(data=[go.Pie(labels=['Keren','Rate'],
                             values=[sum(df['keren']),sum(df['rate'])])])
    pie.update_layout(autosize=False,width=600,height=450,)
    pie.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=16,
                      marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    pie.show()
    
#Line Chart
    fig = px.line(df, x="year", y="debt", title='Debt', markers=True
                 #,text="debt"
                 )
    fig.update_layout(autosize=False,width=600,height=450,)
    fig.update_traces(textposition="top right")
    fig.show()

# Bar Chart
    fig2, ax = plt.subplots()
    plt.rcParams['figure.figsize'] = (8, 6)
    ax.bar(df['year'], df['keren'], width, label='Keren')
    ax.bar(df['year'], df['rate'], width, label='Rate')

    ax.set_ylabel('NIS')
    ax.set_title('Keren & Rate distribution per year')
    ax.legend()
    sns.set_theme()

    plt.show()

    df

In [488]:
# Zamud
years = 10
debt = 100000
r = 0.0111
Loan = debt

inf = 0.03

df = pd.DataFrame(columns=['year','pmt_m','pmt', 'keren','rate', 'debt'])

df.loc[len(df.index)] = [0,0, 0, 0,0,debt]

for n in range(1,years+1):
    pmt_m = -round(npf.pmt((r+inf)/12, years*12, Loan+(debt*inflation),0,0),2)
    pmt = -round(npf.pmt(r+inf, years, Loan+(debt*inflation),0,0),2)
    keren = -round(npf.ppmt(r+inf, n,years, Loan,0,0),2)
    rate = -round(npf.ipmt(r+inf, n,years, Loan,0,0).item(),2)
    debt = round(debt - keren,2)#*(1 + inflation)


    df.loc[len(df.index)] = [n,pmt_m,pmt,keren,rate,debt]
df

,year,pmt_m,pmt,keren,rate,debt
0,0,0,0,0,0,100000
1,1.0,1048.22,12768.6,8286.7,4110.0,91713.3
2,2.0,1045.69,12737.78,8627.28,3769.42,83086.02
3,3.0,1043.05,12705.7,8981.87,3414.84,74104.15
4,4.0,1040.31,12672.29,9351.02,3045.68,64753.13
5,5.0,1037.46,12637.52,9735.35,2661.35,55017.78
6,6.0,1034.48,12601.31,10135.47,2261.23,44882.31
7,7.0,1031.39,12563.62,10552.04,1844.66,34330.27
8,8.0,1028.17,12524.38,10985.73,1410.97,23344.54
9,9.0,1024.81,12483.52,11437.24,959.46,11907.3


In [509]:
# Zamud
years = 10
Loan = 100000
r = 0.0111
debt = Loan

df = pd.DataFrame(columns=['year','pmt_m','pmt', 'keren','rate', 'debt'])

df.loc[len(df.index)] = [0,0, 0, 0,0,debt]

for n in range(1,years+1):
    
    if n <=3:
        praim = 0.03
    elif n <= 5:
        praim = 0.05
    else:
        praim = 0.06
        
    #pmt_m = -round(npf.pmt((r+praim)/12, years*12, Loan,0,0),2)
    #pmt = -round(npf.pmt(r+praim, years, Loan,0,0),2)
    
    keren = -round(npf.ppmt(r, n,years, Loan,0,0),2)
    rate = -round(npf.ipmt(r+praim, n,years, Loan,0,0).item(),2)
    pmt = keren + rate
    pmt_m = round(pmt/12,2)
    debt = round(debt - keren,2)#*(1 - praim)


    df.loc[len(df.index)] = [n,pmt_m,pmt,keren,rate,debt]
df

,year,pmt_m,pmt,keren,rate,debt
0,0,0,0,0,0,100000
1,1.0,1033.06,12396.7,8286.7,4110.0,91713.3
2,2.0,1033.06,12396.7,8627.28,3769.42,83086.02
3,3.0,1033.06,12396.71,8981.87,3414.84,74104.15
4,4.0,1138.14,13657.7,9017.44,4640.26,65086.71
5,5.0,1138.14,13657.69,9568.4,4089.29,55518.31
6,6.0,1192.52,14310.23,10150.71,4159.52,45367.6
7,7.0,1192.52,14310.23,10872.43,3437.8,34495.17
8,8.0,1192.52,14310.23,11645.46,2664.77,22849.71
9,9.0,1192.52,14310.23,12473.45,1836.78,10376.26
